# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
print(OPENAI_API_KEY)  

In [ ]:
pip install --upgrade pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Data.csv')

In [ ]:
df.head()

## LLMChain

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [ ]:
prompt = ChatPromptTemplate.from_template("Can you write a compelling product description for a product called '{product_name}'? Please highlight its key features and benefits.")

In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
print(chain.run(product))

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(  "Come up with a creative and unique idea for a new product in the {category} industry. Include its name, purpose, and what makes it innovative.")

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(  "Based on the following product idea:\n\n{product_idea}\n\nWrite a compelling product description for a website. Make it engaging, highlight key features, and aim to attract potential customers.")
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

**Repeat the above twice for different products**

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
 "Translate the following product review into English:\n\n{review}"

)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
 "Summarize the following product review in 1–2 sentences, focusing on the customer's main opinion and experience:\n\n{translated_review}"

)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary")


In [ ]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
 "Translate the following product review into {target_language}:\n\n{review}"

)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="English_Review"
                      )


In [ ]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
   "Based on the following translated review:\n\n{translated_review}\n\nAnd this summary of the review:\n\n{summary}\n\nWrite a friendly follow-up message thanking the customer for their feedback and optionally asking a relevant follow-up question."

)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
from langchain.chains import SequentialChain
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review", "target_language"],
    output_variables=["translated_review", "summary", "followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain({
    "review": review,
    "target_language": target_language
})

**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [ ]:
# Product 1: Waterproof Phone Pouch
product1 = "Waterproof Phone Pouch"
print(chain.run(product1))

# Product 2: Luxury Air Mattress
product2 = "Luxury Air Mattress"
print(chain.run(product2))


In [ ]:
# Category 1: Electronics
category1 = "Electronics"
print(overall_simple_chain.run(category1))


# Category 2: Fitness Gear
category2 = "Fitness Gear"
print(overall_simple_chain.run(category2))



In [ ]:
# Corrected Code (Define target_language first)
target_language = "French"  

# Review 1 (Index 1: Waterproof Phone Pouch Review)
review1 = df.Review[1]
result1 = overall_chain({"review": review1, "target_language": target_language})
print("Translated Review:", result1["translated_review"])
print("Summary:", result1["summary"])
print("Follow-Up:", result1["followup_message"])


# Review 2 (Index 2: Luxury Air Mattress Review)
review2 = df.Review[2]
result2 = overall_chain({"review": review2, "target_language": "German"})
print("Translated Review:", result2["translated_review"])
print("Summary:", result2["summary"])
print("Follow-Up:", result2["followup_message"])


In [ ]:
# History Question
history_q = "What caused the fall of the Roman Empire?"
print(chain.run(history_q))

# Computer Science Question
cs_q = "Explain recursion in programming."
print(chain.run(cs_q))
